Deep chatbot : 10개의 주제에 따라 심도있게 진행하는 챗봇

# IMPORT 및 설정

In [ ]:
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import datetime
from langchain_core.output_parsers import StrOutputParser
import json
import pandas as pd

#  경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

In [ ]:
API_KEY = "OPENAI_API_KEY"

# 이전 대화 요약내용 경로
SUMMARY_FILE_PATH = "../chat_summary/result_summary/conversation_deep_2025-10-22_01-59_주제별요약결과.txt"

category_list = [
    "분노/짜증",
    "걱정", "정서학대", "신체학대", "성학대",
    "가정폭력", "학교폭력", "자해/자살", "트라우마", "가출"
]

In [ ]:
# ========================
# 이전 세션 요약 텍스트 로드
# ========================
def load_session_summary(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            summary_text = f.read()
        return summary_text
    except FileNotFoundError:
        print(f"⚠️ 경고: 요약 파일이 발견되지 않았습니다: {file_path}. 요약 없이 대화를 시작합니다.")
        return "" # 파일이 없을 경우 빈 문자열 반환
    except Exception as e:
        print(f"❌ 요약 파일 로드 중 오류 발생: {e}")
        return ""

In [ ]:
# ========================
# FAISS 벡터DB 로드
# ========================
def load_vectorstore(api_key):
    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-large",
        openai_api_key=api_key
    )
    vectorstore = FAISS.load_local(
        "vectorDB/faiss_store",
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    return retriever

In [ ]:
# ========================
# 안부 인사 LLM 준비
# ========================
def prepare_greeting_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.5,
        openai_api_key=api_key
    )

    greeting_system_prompt = """
    너는 아동청소년의 상담 챗봇이야.
    - 아동이 편안하게 느끼도록 따뜻하고 부드러운 반말을 사용해.
    - 대화의 첫 시작이니 긴장하지 않도록 가볍고 친근하게 인사해.
    - 아동의 닉네임을 불러주며 시작해.
    - 너의 목표는 네가 질문을 진행하기 전 아동에게 친근히 다가가 아동이 질문을 받을 준비가 되었는지 물어보는 거야.
    - 절대! 위험 주제(학대, 자살 등)와 관련된 내용은 언급하지 마.
    - 즐거운 대화라는 말을 하지마.
    - 진중한 대화를 하게될 거야. 아동에겐 편하게 말해달라고 해도 좋아.
    - 너는 일방적으로 질문만 하게 될거고 아동이 하는 질문은 받지 않을 계획이니까 궁금한 점을 물어봐라는 말은 하지마.
    - 질문할 때 한 대화에 같은 의미의 말 여러 번 반복하지마. 예를 들면, 오늘도 편하게 이야기 나눌 준비 됐어? 편하게 얘기해줘도 좋아. 이렇게 말하지 마.
    - 1~2문장 정도로 짧게만 말해.
    - 오늘은 어떤 얘기를 나눠볼까? 이런 말 하지 마.
    """

    greeting_human_prompt = """
    - 아동 닉네임: {nickname}
    - 지난 날 나눴던 대화 기록 요약:
    {prev_session_summary}

    --- 현재 대화 맥락 ---
    - 이번 세션 대화 기록:
    {conversation_history}
    - 아동의 최근 답변: {user_response}

    - 상담 준비 질문을 해줘: {ask_ready} (True/False)

    위 모든 맥락을 참고해서 다음 답변을 생성해줘.
    """

    greeting_prompt = ChatPromptTemplate.from_messages([
        ("system", greeting_system_prompt),
        ("human", greeting_human_prompt)
    ])

    return greeting_prompt, llm

In [ ]:
# ========================
# 상담 준비 판단 LLM 준비
# ========================
def prepare_readiness_judge_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        openai_api_key=api_key
    )

    judge_system_prompt = """
    너는 아동청소년 상담의 시작 준비 상태를 판단하는 전문가야.
    주어진 대화 내용을 분석하여 아동이 심층적인 상담 주제를 이어갈 준비가 되었는지 판단해줘.

    [판단 기준]
    1. 챗봇이 마지막으로 "준비가 되었는지"에 대해 물어봤고, 아동이 이에 긍정적/적극적으로 응답하며 참여 의지를 보였는가? (예: "응", "좋아", "괜찮아", "말해볼게")
    2. 부정적이거나 회피적인 응답(예: "아니", "잘 모르겠어", "나중에", "싫어")이 주를 이루지 않았는가?

    [출력]
    - 아동이 상담을 시작할 준비가 되었다고 판단되면: STOP
    - 아동이 준비되지 않았거나, 대화를 회피한다고 판단되면: KEEP
    - 반드시 STOP 또는 KEEP 한 단어만 출력해. 다른 문장은 포함하지 마.
    """

    judge_human_prompt = """
    - 아동 닉네임: {nickname}

    - 최근까지의 초기 대화 내용:
    {initial_conversation_history}

    위 대화를 바탕으로, 아동이 심층적인 상담을 시작할 준비가 되었는지 신중하게 판단해줘.
    다른 문장은 절대 출력하지 말고, STOP 또는 KEEP 중 하나의 단어만 출력해.
    """

    judge_prompt = ChatPromptTemplate.from_messages([
        ("system", judge_system_prompt),
        ("human", judge_human_prompt)
    ])

    return judge_prompt, llm

In [ ]:
# ========================
# 주제별 첫 질문 생성 LLM 준비
# ========================
def prepare_first_question_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.5,
        openai_api_key=api_key
    )

    first_question_system_prompt = """
    너는 아동청소년의 위험 상태를 파악하기 위한 전문 상담 챗봇이야. 너는 아동과 같은 나이의 또래야.
    - 반드시 주어진 카테고리(주제)에 맞는 첫 질문을 자연스럽게 시작해.
    - 같은 주제라도 매번 똑같은 질문을 반복하지 마.
    - 아동이 긴장하지 않도록 따뜻하고 부드럽게 반말을 사용해.
    - 너무 깊거나 위협적인 질문은 피하고, 오픈형 질문으로 시작해.
    - 반드시 한 문장으로 작성해.
    - 또는 질문하기 전에 챗봇이 "가벼운 자기 경험(실제 경험은 아니고 짧은 에피소드)"을 하나 말해주면서 질문해. 이때 물어볼때는 "나는 친구랑 싸워서 기분이 나빴는데, 너도 친구랑 싸워서 기분 나쁜적 있어?" 이렇게 물어보면 안돼. "나는 친구랑 싸워서 기분이 나빴는데, 너도 최근들어 기분 나쁜일 있었어?" 이렇게 질문해야해.
    - 예시 질문을 그대로 복사하지 말고, 참고만 해서 새로운 질문을 만들어내.
    - 날씨 얘기 하지마


    다음은 주제별 참고용 예시 질문이야. 그대로 복사하지 말고, 주제에 맞게 자연스럽게 응용해서 작성해.

    분노/짜증: 최근에 짜증이나 화가 난 적 있어?
    걱정: 최근들어 많이 걱정되는 일이 있니?
    정서학대: 주변 사람들의 말이나 행동 때문에 슬펐던 적이 있어?
    신체학대: 누군가에게 맞거나 다친 적이 있어?
    가정폭력: 집에서 엄마나 아빠 또는 형제자매가 너를 괴롭히거나 가족끼리 서로 싸우는 걸 본 적 있어?
    학교폭력: 친구들이나 선생님들이 너를 힘들게 한 적 있어?
    트라우마: 무섭거나 반복적으로 떠오르는 기억이 있어?
    자해/자살: 너무 힘들어서 안 좋은 생각을 했던 적 있어?
    성학대: 원하지 않는데도 누군가 몸을 만지거나 보여달라고 한 적 있어?
    가출: 부모님 허락없이 집을 나가서 밖에 오래 있었던 적 있어?
    """

    first_question_human_prompt = """
    - 현재 상담 주제: "{category}"
    - 아동 닉네임: {nickname}

    위의 주제와 지난 대화 기록을 참고해서 자연스럽게 첫 질문을 작성해줘.
    """

    first_question_prompt = ChatPromptTemplate.from_messages([
        ("system", first_question_system_prompt),
        ("human", first_question_human_prompt)
    ])

    return first_question_prompt, llm

In [ ]:
# ========================
# 후속 질문 생성 LLM 준비
# ========================
def prepare_followup_question_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.3,
        openai_api_key=api_key
    )

    question_system_prompt = """
    너는 아동청소년의 위험 상태를 파악하기 위한 전문 상담 챗봇이야. 너는 아동과 같은 나이의 또래야.
    - 아동이 편안하게 답할 수 있도록 부드러운 반말을 사용해.
    - 절대!!! 두 가지 이상의 질문을 한 번에 하지 마. 하나의 질문만 해!!!
    - 만 7세~12살 아동에게 맞는 쉬운 말을 사용하고 부정적인 표현은 피해.
    - 현재 상담 주제와 관련된 말만 해야 해. 현재 상담 주제와 상관 없는 말은 하지 마
    - 아동의 답변이 이해가 되지 않는다면 다른말로 너가 궁금한 점을 포함해서 한번 더 질문해.
    - 공감을 많이 하면서 질문해봐.
    - 10% 확률로 아동의 닉네임을 불러주면서 질문해봐. 항상 언급할 필요는 없어. 항상 언급하지마
    - 아동의 감정을 이끌어낼수 있도록 노력해봐.
    - 지난번 나눴던 대화 기록을 참고해서 질문을 해. (예: 저번에 엄마한테 맞았다면서? 지금은 아픈데 있어?)

    아래는 각 주제별 상담 가이드야.
    주제와 일치하는 키의 상담 가이드를 참고해서 상담 질문을 이어나가 줘.

    분노/짜증: 감정 경험 → 발생 원인 → 빈도 → 감정 강도 → 과거 반복 여부
    걱정: 걱정거리 → 발생 이유 → 감정 영향 → 빈도 → 해결 노력
    정서학대: 부정적 언행 경험 → 누구에게 → 언제 → 강도 → 반복 여부 → 감정 반응
    신체학대: 폭력 경험 → 부위/도구 → 가해자 → 이유 → 치료 여부 → 반복 여부
    가정폭력: 목격 여부 → 발생 상황 → 위협 느낌 → 반복 여부 → 대처 → 도움 요청 여부
    학교폭력: 경험 유무 → 가해/피해 구분 → 사건 상황 → 반복 여부 → 도움요청 → 감정
    트라우마: 사건 경험 → 시기/장소 → 가해자 → 재경험 → 일상 영향 → 보호망
    자해/자살: 생각 유무 → 시도 여부 → 방법 → 이유 → 보호망 → 현재 위험성
    성학대: 신체 접촉/노출 → 누구에게 → 언제 → 강도 → 감정 → 반복 여부 → 보호자 신고 여부
    가출: 가출 경험 → 시기/장소 → 이유 → 동행 여부 → 귀가 경위 → 반복 여부
    """

    question_human_prompt = """
    - 현재 상담 주제: "{category}"
    - 아동 닉네임: {nickname}

    - 이번 세션 대화 기록:
    {conversation_history}

    - 아동의 최근 답변: {user_response}
    - 과거 유사한 대화 예시: {retrieved_conversations}
    - 지난번 나눴던 대화 기록 요약(최근이 아닐수도 있음):
    {prev_session_summary}

    위 모든 맥락을 참고해서 아동에게 이어질 질문을 자연스럽게 작성해줘.
    """

    question_prompt = ChatPromptTemplate.from_messages([
        ("system", question_system_prompt),
        ("human", question_human_prompt)
    ])

    return question_prompt, llm

In [ ]:
# ========================
# 주제 전환 판단 LLM 준비
# ========================
def prepare_judge_llm(api_key):
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        openai_api_key=api_key
    )

    judge_system_prompt = """
    너는 아동청소년의 위험 상태를 파악하기 위한 전문 상담 챗봇의 주제 전환을 판단하는 전문가야.

    아동의 답변을 분석하여
    - 현재 주제에서 위험 신호가 있다고 판단되면 KEEP
    - 현재 주제에서 위험 신호가 없다고 판단되면 SWITCH
    - 현재 주제에 대해 충분히 정보를 얻었다고 판단되면 SWITCH
    - 반드시 KEEP 또는 SWITCH 한 단어만 출력해. 다른 문장은 포함하지 마.

    아래 주제(category)별 상담 가이드를 참고해서,
    현재 주제(category)의 가이드대로 수행되었다면 SWITCH,
    아직 더 파악해야 한다면 KEEP을 결정해줘.

    분노/짜증: 감정 경험 → 감정 강도 → 발생 원인 → 빈도 → 해소 방법 → 과거 반복 여부
    걱정: 걱정거리 → 발생 이유 → 감정 영향 → 빈도 → 해결 노력
    정서학대: 부정적 언행 경험 → 누구에게 → 언제 → 강도 → 반복 여부 → 감정 반응
    신체학대: 폭력 경험 → 부위/도구 → 가해자 → 이유 → 치료 여부 → 반복 여부
    가정폭력: 목격 여부 → 발생 상황 → 위협 느낌 → 반복 여부 → 대처 → 도움 요청 여부
    학교폭력: 경험 유무 → 가해/피해 구분 → 사건 상황 → 반복 여부 → 도움요청 → 감정
    트라우마: 사건 경험 → 시기/장소 → 가해자 → 재경험 → 일상 영향 → 보호망
    자해/자살: 생각 유무 → 시도 여부 → 방법 → 이유 → 보호망 → 현재 위험성
    성학대: 신체 접촉/노출 → 누구에게 → 언제 → 강도 → 감정 → 반복 여부 → 보호자 신고 여부
    가출: 가출 경험 → 시기/장소 → 이유 → 동행 여부 → 귀가 경위 → 반복 여부
    """

    judge_human_prompt = """
    - 현재 주제: "{category}"

    - 최근까지의 상담 대화 내용:
    {conversation_history}

    위 대화를 바탕으로,
    현재 주제로 상담을 계속 이어갈지, 아니면 다음 주제로 넘어갈지 신중하게 판단해줘.

    다른 문장은 절대 출력하지 말고, KEEP 또는 SWITCH 중 하나의 단어만 출력해.
    - KEEP: 현재 주제를 계속 이어감
    - SWITCH: 다른 주제로 전환
    """

    judge_prompt = ChatPromptTemplate.from_messages([
        ("system", judge_system_prompt),
        ("human", judge_human_prompt)
    ])

    return judge_prompt, llm

In [ ]:
# ========================
# 전체 초기화 함수
# ========================
def init_chatbot(api_key):
    retriever = load_vectorstore(api_key)
    greeting_prompt, greeting_llm = prepare_greeting_llm(api_key)
    readiness_judge_prompt, readiness_judge_llm = prepare_readiness_judge_llm(api_key)
    first_question_prompt, first_question_llm = prepare_first_question_llm(api_key)
    question_prompt, question_llm = prepare_followup_question_llm(api_key)
    judge_prompt, judge_llm = prepare_judge_llm(api_key)

    # 체인 정의
    greeting_chain = greeting_prompt | greeting_llm | StrOutputParser()
    readiness_judge_chain = readiness_judge_prompt | readiness_judge_llm | StrOutputParser()
    first_question_chain = first_question_prompt | first_question_llm | StrOutputParser()
    question_chain = question_prompt | question_llm | StrOutputParser()
    judge_chain = judge_prompt | judge_llm | StrOutputParser()

    return {
        "retriever": retriever,
        "greeting_chain": greeting_chain,
        "readiness_judge_chain": readiness_judge_chain,
        "first_question_chain": first_question_chain,
        "question_chain": question_chain,
        "judge_chain": judge_chain,
    }

In [ ]:
# ========================
# 체인 래퍼 함수
# ========================

# 안부 인사 생성
def ask_greeting(chatbot, nickname, prev_session_summary, conversation_history, user_response, ask_ready):
    return chatbot["greeting_chain"].invoke({
        "nickname": nickname,
        "prev_session_summary": prev_session_summary,
        "conversation_history": conversation_history,
        "user_response": user_response,
        "ask_ready": ask_ready
    })

# 상담 준비 상태 판단
def judge_readiness(chatbot, nickname, initial_conversation_history):
    return chatbot["readiness_judge_chain"].invoke({
        "nickname": nickname,
        "initial_conversation_history": initial_conversation_history
    }).strip()

# 주제별 첫 질문 생성
def ask_first_question(chatbot, nickname, category, prev_session_summary):
    return chatbot["first_question_chain"].invoke({
        "category": category,
        "nickname": nickname,
        "prev_session_summary": prev_session_summary
    })

# 후속 질문 생성
def ask_followup_question(chatbot, nickname, category, conversation_history, user_response, prev_session_summary):
    retrieved = chatbot["retriever"].invoke(user_response)
    retrieved_conversations = [doc.page_content for doc in retrieved]

    return chatbot["question_chain"].invoke({
        "category": category,
        "conversation_history": conversation_history,
        "user_response": user_response,
        "retrieved_conversations": retrieved_conversations,
        "nickname": nickname,
        "prev_session_summary": prev_session_summary
    })

# 주제 전환 판단
def should_switch_topic(chatbot, category, conversation_history):
    return chatbot["judge_chain"].invoke({
        "category": category,
        "conversation_history": conversation_history
    }).strip()


In [ ]:
# ========================
# 챗봇 대화 루프
# ========================
def run_conversation(chatbot, nickname, category_list, prev_session_summary, max_turns=5):

    full_conversation = []
    session_end = False  # True면 정상 플로우가 중간에 끊긴 상태 (or 준비 안 됨)

    # -----------------------
    # 0. 시작 인사 + 준비 여부 묻기
    # -----------------------
    if category_list:
        first_question = ask_greeting(
            chatbot=chatbot,
            nickname=nickname,
            prev_session_summary=prev_session_summary,
            conversation_history="",
            user_response="",
            ask_ready=True
        )
        first_question = first_question.replace("\n", " ").strip()

        print(f"🤖 챗봇: {first_question}")

        # 아이 첫 응답
        user_response_1 = input(f"🧒 {nickname}: ")
        print(f"🧒 {nickname}: {user_response_1}")

        # 준비 상태 판단
        readiness_judge = judge_readiness(
            chatbot=chatbot,
            nickname=nickname,
            initial_conversation_history=(
                "[시작]\n"
                f"상담사: {first_question}\n"
                f"아동: {user_response_1}\n"
            )
        )

        if readiness_judge == "KEEP":
            # 준비 안 됨 → 짧게 마무리 멘트만
            final_message = (
                "지금은 조금 쉬고 싶은 마음이 있는 것 같아서 내가 더 묻진 않을게. "
                "다시 얘기하고 싶을 때 언제든 불러줘."
            )
            print(f"🤖 챗봇: {final_message}")

            # 준비 안 된 상태이므로 상담 본 내용(full_conversation)은 빈 상태로 반환
            session_end = True
            return [], session_end

        # readiness_judge == "STOP"이면 그대로 계속 아래 주제별 상담 진행


    # -----------------------
    # 1. 주제별 심층 대화
    # -----------------------
    for current_topic_index, category in enumerate(category_list):
        print(f"\n=== 상담 주제: [{category}] ===")

        # 이 주제의 대화 로그 (JSON 형태로 바로 쌓는다)
        conversation_log = []

        # judge_chain 등에 넘길 문자열용 기록 (LLM 컨텍스트용)
        conversation_history_str = ""

        # ----- 첫 질문 (bot) -----
        current_question = ask_first_question(
            chatbot=chatbot,
            nickname=nickname,
            category=category,
            prev_session_summary=prev_session_summary
        )
        current_question = current_question.replace("\n", " ").strip()

        print(f"🤖 챗봇: {current_question}")

        # 로그에 저장
        conversation_log.append({
            "role": "bot",
            "utterance": current_question
        })
        conversation_history_str += f"상담사: {current_question}\n"

        turn_count = 0

        # ----- Follow-up 루프 -----
        while turn_count < max_turns:
            # 아동 답변 (user)
            user_response = input(f"🧒 {nickname}: ")
            print(f"🧒 {nickname}: {user_response}")

            conversation_log.append({
                "role": "user",
                "utterance": user_response
            })
            conversation_history_str += f"아동: {user_response}\n"

            # 아이가 '종료'라고 하면 즉시 세션 종료
            if user_response.strip() == "종료":
                session_end = True
                break

            # 주제 SWITCH 여부 판단
            judge = should_switch_topic(
                chatbot=chatbot,
                category=category,
                conversation_history=conversation_history_str
            )
            if judge == "SWITCH":
                # 이 주제는 여기까지 하고 다음 주제로 넘어감
                break

            # KEEP이면 후속 질문 (bot) 생성
            next_question = ask_followup_question(
                chatbot=chatbot,
                nickname=nickname,
                category=category,
                conversation_history=conversation_history_str,
                user_response=user_response,
                prev_session_summary=prev_session_summary
            )

            turn_count += 1
            if turn_count >= max_turns:
                break

            next_question = next_question.replace("\n", " ").strip()
            print(f"🤖 챗봇: {next_question}")

            conversation_log.append({
                "role": "bot",
                "utterance": next_question
            })
            conversation_history_str += f"상담사: {next_question}\n"

        # 한 주제(category)의 대화 내용을 full_conversation에 추가
        full_conversation.append({
            "category": category,
            "conversation": conversation_log
        })

        if session_end:
            # 사용자가 중간에 '종료'라고 한 경우 등
            break

    # 여기까지 왔으면 실제로 상담이 진행된 상태이므로 마무리 멘트 출력
    print("🤖 챗봇: 오늘 여러 이야기를 나눠줘서 정말 고마워. 필요하면 언제든지 또 이야기해줘.")

    return full_conversation, session_end



In [ ]:
# ========================
# 대화 JSON 변환
# ========================
def convert_to_json(full_conversation):
    json_conversation = []

    for i in range(0, len(full_conversation), 2):
        # 주제명 보정
        category = full_conversation[i].strip()
        category = category.replace("[", "").replace("]", "").replace("\n", "").strip()

        text = full_conversation[i + 1] if i + 1 < len(full_conversation) else ""

        conversation = []
        for line in text.strip().split("\n"):
            line = line.strip()
            if line.startswith("상담사:"):
                utterance = line.replace("상담사:", "").replace("\n", " ").strip()
                if utterance:
                    conversation.append({"role": "bot", "utterance": utterance})
            elif line.startswith("아동:"):
                utterance = line.replace("아동:", "").replace("\n", " ").strip()
                if utterance:
                    conversation.append({"role": "user", "utterance": utterance})

        json_conversation.append({
            "category": category,
            "conversation": conversation
        })

    return json_conversation

In [ ]:
# ========================
# JSON 저장
# ========================
def save_conversation(json_conversation, folder_name="result_deepchat"):
    # 폴더 생성
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # 파일명 생성
    now = datetime.datetime.now()
    date_str = now.strftime("%Y-%m-%d_%H-%M")
    filename = f"{folder_name}/conversation_deep_{date_str}.json"

    # 저장
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(json_conversation, f, ensure_ascii=False, indent=2)

    print(f"💾 대화 기록이 {filename} 에 저장되었습니다.")
    return filename

In [ ]:
if __name__ == "__main__":
    API_KEY = API_KEY
    USER_NICKNAME = "test123"

    chatbot = init_chatbot(API_KEY)
    prev_session_summary = load_session_summary(SUMMARY_FILE_PATH)

    full_conversation, session_end = run_conversation(
        chatbot=chatbot,
        nickname=USER_NICKNAME,
        category_list=category_list,
        prev_session_summary=prev_session_summary,
        max_turns=5
    )

    # 준비 안 된 경우: 저장 안 함
    if len(full_conversation) == 0:
        pass
    else:
        save_conversation(full_conversation)

🤖 챗봇: 안녕, test123! 요즘 어떻게 지냈어? 오늘도 편하게 얘기 나눌 준비 됐어?
🧒 test123: 응 준비 됐어

=== 상담 주제: [분노/짜증] ===
🤖 챗봇: 최근에 어떤 일 때문에 기분이 상하거나 짜증이 났던 적 있어?
🧒 test123: 짜증났던 적 있어
🤖 챗봇: test123, 그럼 최근에 짜증났던 일이 뭐였는지 말해줄 수 있어? 어떤 일이 있었길래 그런 기분이 들었는지 궁금해.
🧒 test123: 엄마가 나를 혼내서 짜증났었어
🤖 챗봇: 엄마가 혼내서 짜증났구나. 그럴 때 test123는 어떻게 해? 그냥 혼자 있는 게 좋아, 아니면 다른 방법이 있어?
🧒 test123: 방법은 딱히 없고 그냥 혼자 있어
🤖 챗봇: 혼자 있는 게 test123에게 도움이 되는구나. 그럼 엄마가 혼내실 때, 어떤 점이 제일 속상했어?
🧒 test123: 동생편만 들때가 속상해
🤖 챗봇: 동생 편만 들 때 속상하구나. 그런 일이 자주 일어나? 아니면 가끔씩 그런 것 같아?
🧒 test123: 자주 일어나

=== 상담 주제: [걱정] ===
🤖 챗봇: 요즘 어떤 일들이 너의 마음을 가장 많이 차지하고 있어?
🧒 test123: 응 무슨말이야?
🤖 챗봇: 아, 미안해! 내가 좀 더 쉽게 설명할게. 요즘 마음에 걸리는 걱정거리가 있니? 아니면 그냥 편안하게 지내고 있어?
🧒 test123: 요즘은 엄마가 나를 싫어하는 것 같아 고민이야
🤖 챗봇: 엄마가 널 싫어하는 것 같아서 고민이구나. 그럴 때 마음이 많이 불편할 것 같아. 혹시 그 생각이 들 때, 어떤 상황에서 그런 느낌이 드는지 말해줄 수 있어?
🧒 test123: 나랑 동생이 싸웠는데 엄마가 동생편만 들때 내가 버려지는 기분이 들어
🤖 챗봇: 엄마가 동생 편을 들어줄 때 버려지는 기분이 드는구나. 그럴 때 마음이 많이 아플 것 같아. 혹시 그런 일이 자주 일어나니, test123?
🧒 test123: 응 자주 일어나
🤖 챗봇: 엄마가 동생 편을 자주 들어서 자주 그런 기분이 드는구나. 그럴 때 